# Assignment 7

### Part A

In [77]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [78]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [79]:
def expected_return(P, rf, mkt_rp):
    # Write your code here

    length = len(P["Beta"])

    # Find total value so we can find weighting of each
    total_value = 0 
    for i in range(length):
        total_value += P["Amount Invested"][i]
    
    # βp​ = weight1 × β1 ​+ weight2 ​× β2 + ...
    beta = 0 
    for i in range(length):
        beta += (P["Amount Invested"][i] / total_value)* P["Beta"][i]

    return rf + beta * mkt_rp # CAPM

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [81]:
def volatility(P, cor):
    # Write your code here
    length = len(P["Beta"])

    # Find total value so we can find weighting of each
    total_value = 0 
    for i in range(length):
        total_value += P["Amount Invested"][i]
    
    weighting = []
    for i in range(length):
        weighting.append(P["Amount Invested"][i] / total_value)

    # Self note: Google the volatility of a three stock portfolio for strategy (and expand on that)
    variance = 0
    for i in range(length):
        variance += weighting[i] * weighting[i] * P["Volatility"][i] * P["Volatility"][i]
    
    for i in range(length):
        for j in range(i + 1, length):
            variance += 2 * weighting[i] * weighting[j] * cor * P["Volatility"][i] * P["Volatility"][j]

    volatility = variance ** 0.5
    return volatility


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [83]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here

    # Current volality is mkt_vol but we want a volatility of vol
    # x * mkt_vol = vol
    x = vol / mkt_vol

    return rf + x * mkt_rp


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [85]:
def CAPM_beta(return_path, factors_path):

    df_return = pd.read_csv(return_path)
    df_factor = pd.read_csv(factors_path)

    #Write your code here
    output = pd.DataFrame(data = {
        "CUSIP" : [] ,
        "EXP_RETURN" : [],
        "BETA" : [],
    })

    df_return['date'] = (df_return['year'].astype(str) + df_return['month'].astype(str).str.zfill(2)).astype(int)
    df_merge = pd.merge(df_return, df_factor, on='date')

    
    value_counts = df_merge["CUSIP"].value_counts()

    for index, value in value_counts.items():
        exp_ret = 0
        filtered_df = df_merge[df_merge['CUSIP'].isin([index])].copy()
        filtered_df["Numeric"] = pd.to_numeric(filtered_df["RET"], errors ='coerce') 
        exp_ret += filtered_df['Numeric'].sum()
        exp_ret = exp_ret / value

        cov_matrix = filtered_df[['Numeric', 'Mkt-RF']].cov().values
        beta = cov_matrix[0, 1] / filtered_df['Mkt-RF'].var()

        # Adjust beta for the risk-free rate
        beta_adjusted = beta * (1 + (filtered_df['RF'].mean() / 100))

        data = pd.DataFrame(data = {
        "CUSIP" : [index] ,
        "EXP_RETURN" : [exp_ret],
        "BETA" : [beta_adjusted],
        })
        output = pd.concat([output, data])


    
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [87]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    
    c = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    return c[1]